In [ ]:
import re
from collections import defaultdict
import rich
import sys

In [ ]:
__cmd__ = "help"
hostname = "" #Hostname used when adding or removing
dest = "127.0.0.1" #Destination ip for traffic that should come from hostname
dry = True #Dry run will prevent /etc/hosts updates
allowBypassOfDotLocalRestriction = False # By Default only allow override of domains ending in .local

In [ ]:
def loadHosts():
    try:
        with open('/etc/hosts', 'r') as f:
            data = f.read()
            return data
    except Exception as e:
        print(e)

In [ ]:
def parseHosts(data):
    hosts = defaultdict(list)
    for line in data.splitlines():
        if line == '':
            continue
        dest, *split = re.split('\\s+', line)
        if dest == '#':
            continue
        hosts[dest] += split
    hosts = dict(hosts)
    return hosts

In [ ]:
__cmd__ = __cmd__.lower()

In [ ]:
if __cmd__ not in ['list', 'add', 'remove']:
    print("""
    Commands Include: help, list, add, remove
    """)
    sys.exit()

In [ ]:
if __cmd__ == "list":
    hosts = parseHosts(loadHosts())
    rich.print(hosts)
    sys.exit()

In [ ]:
hostname = hostname.strip()
if hostname == '':
    print("Hostname Missing")
    sys.exit(1)

In [ ]:
needs_write = False
if not hostname.endswith('.local'):
    if input(f'.local missing, update hostname to {hostname}.local; continue [Y/n]:').lower() != 'y':
        if not allowBypassOfDotLocalRestriction:
            print("Flag must be set to allow for bypass of local restrictions please check help")
            sys.exit()
    else:
        hostname += '.local'

In [ ]:
if __cmd__ == "add":
    if hostname == '':
        print('you must supply a hostname to add a redirect')
        sys.exit(1)
    needs_write = True
    hosts = parseHosts(loadHosts())
    if hostname not in hosts[dest]:
        hosts[dest].append(hostname)

In [ ]:
if __cmd__ == "remove":
    if hostname == '':
        print('you must supply a hostname to remove a redirect')
        sys.exit(1)
    needs_write = True
    hosts = parseHosts(loadHosts())
    if hostname in hosts[dest]:
        hosts[dest].remove(hostname)

In [ ]:
if dry and needs_write:
    print("# PREVIEW (DRY RUN)\n")
    for dest in hosts:
        print(f"{dest}\t{' '.join(hosts[dest])}")

In [ ]:
if not dry:
    if needs_write:
        with open('/etc/hosts', 'w') as f:
            for dest in hosts:
                f.write(f"{dest}\t{' '.join(hosts[dest])}\n")